In [7]:
import sqlite3
import pandas.io.sql as sql

In [8]:
# movie db와 연결

con = sqlite3\
.connect('/Users/grace/workspace/SQLite/movie_ratings.db')

In [9]:
con

In [10]:
users = sql.read_sql_query('SELECT * FROM users;', con)

In [12]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [13]:
movies = sql.read_sql_query('SELECT * FROM movies;', con)

In [14]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings = sql.read_sql_query('SELECT * FROM ratings;', con)

In [16]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 평점 추출해보기

In [17]:
drama_query = """
    SELECT * FROM movies
    WHERE genres
    LIKE '%Drama%';
"""

drama_movies = sql.read_sql_query(drama_query, con)

In [19]:
drama_movies.head()

,movie_id,title,genres
0,4,Waiting to Exhale (1995),Comedy|Drama
1,11,"American President, The (1995)",Comedy|Drama|Romance
2,14,Nixon (1995),Drama
3,16,Casino (1995),Drama|Thriller
4,17,Sense and Sensibility (1995),Drama|Romance


In [20]:
comb_movies_ratings = """
    SELECT * FROM movies
    INNER JOIN ratings
    ON movies.movie_id = ratings.movie_id
    WHERE genres LIKE '%Drama%'
    GROUP BY movies.movie_id;
"""

In [21]:
drama_ratings = sql.read_sql_query(comb_movies_ratings, con)

In [22]:
drama_ratings.head()

,movie_id,title,genres,user_id,movie_id,rating,timestamp
0,4,Waiting to Exhale (1995),Comedy|Drama,6036,4,2,956753920
1,11,"American President, The (1995)",Comedy|Drama|Romance,6036,11,3,956752783
2,14,Nixon (1995),Drama,6016,14,3,956959191
3,16,Casino (1995),Drama|Thriller,6036,16,3,956753965
4,17,Sense and Sensibility (1995),Drama|Romance,6040,17,3,956704584


In [23]:
# movie_id, title, count(rating), avg(rating)

avg_query = """
    SELECT movies.movie_id,
    title,
    count(ratings.rating) as cnt,
    avg(ratings.rating) as avgr
    FROM movies
    INNER JOIN ratings
    ON movies.movie_id=ratings.movie_id
    WHERE genres LIKE '%Drama%'
    GROUP BY movies.movie_id
    ORDER BY avgr DESC;
"""

In [24]:
avg_ratings = sql.read_sql_query(avg_query, con)

In [25]:
avg_ratings.head()

,movie_id,title,cnt,avgr
0,989,Schlafes Bruder (Brother of Sleep) (1995),1,5.00
1,3382,Song of Freedom (1936),1,5.00
2,3607,One Little Indian (1973),1,5.00
3,3245,I Am Cuba (Soy Cuba/Ya Kuba) (1964),5,4.80
4,53,Lamerica (1994),8,4.75


In [26]:
limit_cnt = """
    SELECT movies.movie_id,
    title,
    count(ratings.rating) as cnt,
    avg(ratings.rating) as avgr
    FROM movies
    INNER JOIN ratings
    ON movies.movie_id=ratings.movie_id
    WHERE genres LIKE '%Drama%'
    GROUP BY movies.movie_id
    HAVING cnt > 100
    ORDER BY avgr DESC;
"""

In [27]:
limit_rating = sql.read_sql_query(limit_cnt, con)

In [28]:
limit_rating.head()

,movie_id,title,cnt,avgr
0,2019,Seven Samurai (The Magnificent Seven) (Shichin...,628,4.560510
1,318,"Shawshank Redemption, The (1994)",2227,4.554558
2,858,"Godfather, The (1972)",2223,4.524966
3,527,Schindler's List (1993),2304,4.510417
4,1178,Paths of Glory (1957),230,4.473913


## USING

In [45]:
avg_query = """
    SELECT title, cnt, avg_rating
    FROM(
        SELECT title, 
        count(rating) as cnt,
        avg(rating) as avg_rating
        FROM movies JOIN ratings
        USING(movie_id)
        WHERE movies.genres LIKE '%Drama%'
        GROUP BY movies.movie_id
        HAVING cnt >100
    )
    GROUP BY title
    ORDER BY avg_rating DESC;
"""

# """
# SELECT title, cnt, avg_rating
# FROM(
#     SELECT title, 
#     count(rating) as cnt,
#     AVG(rating) as avg_rating
#     FROM movies JOIN ratings USING (movie_id) 
#     WHERE movies.genres LIKE '%Drama%'
#     GROUP BY ratings.movie_id 
#     HAVING cnt >=100
# )
# GROUP BY title
# ORDER BY avg_rating DESC;
# """

In [46]:
avg2_df = sql.read_sql_query(avg_query, con)

In [47]:
avg2_df.head()

,title,cnt,avg_rating
0,Seven Samurai (The Magnificent Seven) (Shichin...,628,4.560510
1,"Shawshank Redemption, The (1994)",2227,4.554558
2,"Godfather, The (1972)",2223,4.524966
3,Schindler's List (1993),2304,4.510417
4,Paths of Glory (1957),230,4.473913
